In [2]:
import torch

from botorch.fit import fit_gpytorch_model
from botorch.models import SingleTaskGP
from botorch.test_functions import Hartmann
from gpytorch.mlls import ExactMarginalLogLikelihood

neg_hartmann6 = Hartmann(dim=6, negate=True)

In [3]:
train_x = torch.rand(10, 6)
train_obj = neg_hartmann6(train_x).unsqueeze(-1)
model = SingleTaskGP(train_X=train_x, train_Y=train_obj)
mll = ExactMarginalLogLikelihood(model.likelihood, model)
fit_gpytorch_model(mll)

ExactMarginalLogLikelihood(
  (likelihood): GaussianLikelihood(
    (noise_covar): HomoskedasticNoise(
      (noise_prior): GammaPrior()
      (raw_noise_constraint): GreaterThan(1.000E-04)
    )
  )
  (model): SingleTaskGP(
    (likelihood): GaussianLikelihood(
      (noise_covar): HomoskedasticNoise(
        (noise_prior): GammaPrior()
        (raw_noise_constraint): GreaterThan(1.000E-04)
      )
    )
    (mean_module): ConstantMean()
    (covar_module): ScaleKernel(
      (base_kernel): MaternKernel(
        (lengthscale_prior): GammaPrior()
        (raw_lengthscale_constraint): Positive()
        (distance_module): Distance()
      )
      (outputscale_prior): GammaPrior()
      (raw_outputscale_constraint): Positive()
    )
  )
)

In [4]:
from botorch.acquisition import qExpectedImprovement

best_value = train_obj.max()
qEI = qExpectedImprovement(model=model, best_f=best_value)

In [7]:
from botorch.optim import optimize_acqf

new_point_analytic, _ = optimize_acqf(
    acq_function=qEI,
    bounds=torch.tensor([[0.0] * 6, [1.0] * 6]),
    q=5,
    num_restarts=20,
    raw_samples=100,
    options={},
)
new_point_analytic

tensor([[0.1963, 0.1782, 0.7700, 0.5441, 0.0000, 0.3995],
        [0.3535, 0.3192, 0.7376, 0.4917, 0.3215, 0.2968],
        [0.3112, 0.4341, 0.6361, 0.3728, 0.0240, 0.5370],
        [0.1515, 0.2738, 0.5781, 0.2874, 0.2172, 0.1711],
        [0.1506, 0.5259, 0.8371, 0.3554, 0.0000, 0.1550]])